In [7]:
from datetime import datetime
import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
from PIL import Image
import html
from markdownify import markdownify 

endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"

query_txt = """PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT ?textItem ?kuratorLabel ?textUrl
WHERE
{
  <placeholder>
  ?textItem cpsp:P46 ?kuratorStatement. 
  ?kuratorStatement cpsps:P46 ?kuratorItem. 
  ?kuratorItem rdfs:label ?kuratorLabel.
  ?textItem cpsp:P57 ?urlstatement. 
  ?urlstatement cpsps:P57 ?textUrl. 
}"""

query_img ="""
PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT DISTINCT ?itemLabel ?itemDescr ?imgItem ?imgUrl ?publishDate
WHERE
{
  ?imgItem cpsp:P107 ?urlStatement. 
  ?urlStatement cpsps:P107 ?imgUrl. 
  ?imgItem cpsp:P60 ?dateStatement. 
  ?dateStatement cpsps:P60 ?publishDate. 
  ?imgItem cpsp:P6 ?partOfStatement.
  ?partOfStatement cpsps:P6 ?partOfItem.
  <placeholder> 

  FILTER (datatype(?publishDate) = xsd:edtf)
  
  SERVICE wikibase:label {
      bd:serviceParam wikibase:language "en,de".
      ?imgItem rdfs:label ?itemLabel.
      ?imgItem schema:description ?itemDescr.
    }
}
"""

query_graph = """PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT ?x ?y
WHERE
{
  ?a cpsp:P2 ?c.
  ?c cpsps:P2 ?d.
  ?a rdfs:label ?x.
  ?d rdfs:label ?y.

}LIMIT 1"""

query_graph2 = """
SELECT ?a ?b ?c
WHERE
{
  ?a rdfs:label ?c
}LIMIT 100"""


def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_delay(date):
    try:
        date = datetime.datetime.strptime(date, '%a, %d %b %Y %H:%M:%S GMT')
        timeout = int((date - datetime.datetime.now()).total_seconds())
    except ValueError:
        timeout = int(date)
    return timeout

def fetch_image_by_url(url, headers):
    r = requests.get(url, headers=headers, stream=True)
    if r.status_code == 200:
        im = Image.open(r.raw)
        return im
    if r.status_code == 500:
        return None
    if r.status_code == 403:
        return None
    if r.status_code == 429:
        timeout = get_delay(r.headers['retry-after'])
        print('Timeout {} m {} s'.format(timeout // 60, timeout % 60))
        time.sleep(timeout)
        fetch_image_by_url(url, headers)

def get_img(partOfItem_id):
    print("")
    q = ""
    if partOfItem_id:
        q = query_img.replace("<placeholder>", "?partOfStatement cpsps:P6 cps:"+partOfItem_id+".")
    else:
        q = query_img.replace("<placeholder>","")
    results_img = run_query(endpoint_url, q)

    print(':::{.column-page}')
#     print('''
# ::: {.content-visible when-format="pdf"}

# | Right | Left | Default | Center |
# |:------:|:------:|:------:|:------:|
# |
# :::
#     ''')

    count = 0

    for item in results_img["results"]["bindings"]:    
      #print(item)
      #print('Wikibase link: ' + '[' + item['imgItem']['value'] + ']' + '(' + item['imgItem']['value'] + ')' + '\n')
      #print('Title: ' + item['itemLabel']['value'] + '\n')
      #print('Year: ' + item['publishDate']['value'] + '\n')
      #print('Description: ' + html.unescape(item['itemDescr']['value']) + '\n')

      # get image from image URL and display resized version

      alt=item['itemDesc']['value'] if 'itemDesc' in item else ''

      url = item['imgUrl']['value']
      image_path = 'images/' + url.split('/')[-1]
      r = requests.get(url, allow_redirects=True)
      open(image_path, 'wb').write(r.content)

      item_url=item['imgItem']['value'] 
      title=item['itemLabel']['value'] if 'itemLabel' in item else ''
      year=item['publishDate']['value'] if 'publishDate' in item else ''
      #headers = {'User-Agent': 'Ex_Books_conference_bot/0.0 (https://github.com/SimonXIX/Experimental_Books_workshop; ad7588@coventry.ac.uk)'}
      #im = fetch_image_by_url(image_url, headers)
      #im.thumbnail((500, 500), Image.Resampling.LANCZOS)
      #display(im)
      onclickopen = f"openPopover('pop{image_path}')"
      onclickclose = f"closePopover('pop{image_path}')"
      openinfo = f"openInfo('info{item_url}')"
      closeinfo = f"closeInfo('info{item_url}')"


      print("""
::: {.content-visible unless-format='pdf'}
      """)
      print('<div class="outerinfo">')
      print(f'<input type="image" class="img_square" src="{image_path}" popovertarget="pop{image_path}" onclick="{onclickopen},{openinfo}"/>')
      #print('<div class="infohover"></div>')
      print("</div>")
      print(f'<img popover class="img_popover" src="{image_path}" alt="{alt}" id="pop{image_path}" onclick="{onclickclose},{closeinfo}" style="display: none;">')    
      print(f'<div popover class="info-wd btn-group" id="info{item_url}" style="display:none"><a href="{item_url}"><button class="btn btn-light">{title}</button></a><button type="button" class="btn btn-secondary">value</button><button type="button" class="btn btn-secondary">{year}</button></div>')
#print(f'<div class="close-btn" id="close_button_{image_url} onclick="{onclickclose}" style="display:none;"/></div>')
      print("""
:::
      """)
      


      print("""
::: {.content-visible when-format='pdf'}
      """)
      print(f"""
\\begin{{figure}}[H]    
  \\includegraphics[height=10cm]{{{image_path}}}
  \\caption{{{title.replace('&', 'and')}}}
  \\label{{fig:{{{image_path}}}}}
\\end{{figure}}

```{{=latex}}
\\clearpage
```
      """)

      print('''
:::
      ''')
print("""
          
<script>
  function openPopover(imageUrl) {
    var popover = document.getElementById(imageUrl);
    var body = document.querySelector('body');
    // Show the popover
    popover.style.display = 'block';
    body.classList.add("stop-scrolling");
  };

  function closePopover(imageUrl) {
    var popover = document.getElementById(imageUrl);
    var body = document.querySelector('body');
    popover.style.display = 'none';
    body.classList.remove("stop-scrolling");
  };
          
  
  function openInfo(itemUrl) {
    var popover = document.getElementById(itemUrl);
    var body = document.querySelector('body');
    // Show the popover
    popover.style.display = 'flex';
  };
                          
  function closeInfo(itemUrl) {
    var popover = document.getElementById(itemUrl);
    var body = document.querySelector('body');
    popover.style.display = 'none';
  };

          
</script>
""")





# def get_graph():
#     import VizKG.visualize as vkg
#     results_graph1 = run_query(endpoint_url, query_graph)
#     #print(results_graph1)
#     #print('---')
#     results_graph2 = run_query(endpoint_url, query_graph2)
#     #print(results_graph2)

#     chart = vkg(sparql_query=query_graph2, sparql_service_url=endpoint_url, chart='wordcloud')
#     chart.plot()

    #print(f'![{alt}]({image_path}){{height=10cm fig-cap=false}}')



          
<script>
  function openPopover(imageUrl) {
    var popover = document.getElementById(imageUrl);
    var body = document.querySelector('body');
    // Show the popover
    popover.style.display = 'block';
    body.classList.add("stop-scrolling");
  };

  function closePopover(imageUrl) {
    var popover = document.getElementById(imageUrl);
    var body = document.querySelector('body');
    popover.style.display = 'none';
    body.classList.remove("stop-scrolling");
  };
          
  
  function openInfo(itemUrl) {
    var popover = document.getElementById(itemUrl);
    var body = document.querySelector('body');
    // Show the popover
    popover.style.display = 'flex';
  };
                          
  function closeInfo(itemUrl) {
    var popover = document.getElementById(itemUrl);
    var body = document.querySelector('body');
    popover.style.display = 'none';
  };

          
</script>



In [8]:
get_img("")


:::{.column-page}

::: {.content-visible unless-format='pdf'}
      
<div class="outerinfo">
<input type="image" class="img_square" src="images/fmd10005862a.jpg" popovertarget="popimages/fmd10005862a.jpg" onclick="openPopover('popimages/fmd10005862a.jpg'),openInfo('infohttps://computational-publishing-service.wikibase.cloud/entity/Q212')"/>
</div>
<img popover class="img_popover" src="images/fmd10005862a.jpg" alt="" id="popimages/fmd10005862a.jpg" onclick="closePopover('popimages/fmd10005862a.jpg'),closeInfo('infohttps://computational-publishing-service.wikibase.cloud/entity/Q212')" style="display: none;">
<div popover class="info-wd btn-group" id="infohttps://computational-publishing-service.wikibase.cloud/entity/Q212" style="display:none"><a href="https://computational-publishing-service.wikibase.cloud/entity/Q212"><button class="btn btn-light">Knight's Hall & Room 72 - to the west</button></a><button type="button" class="btn btn-secondary">value</button><button type="button" class=

<hr>

In [9]:
#get_graph()